In [9]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil

In [2]:
from architect import Architect
from model_search import Network

In [3]:
tf.enable_eager_execution()

In [4]:
def parse_args():
    parser = argparse.ArgumentParser("cifar")
    parser.add_argument('--data', type=str, default='./data',
                        help='location of the data corpus')
    parser.add_argument('--batch_size', type=int,
                        default=1, help='batch size')
    parser.add_argument('--learning_rate', type=float,
                        default=0.025, help='init learning rate')
    parser.add_argument('--learning_rate_min', type=float,
                        default=0.001, help='min learning rate')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
    parser.add_argument('--weight_decay', type=float,
                        default=3e-4, help='weight decay')
    parser.add_argument('--report_freq', type=float,
                        default=10, help='report frequency')
    parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
    parser.add_argument('--epochs', type=int, default=50,
                        help='num of training epochs')
    parser.add_argument('--init_channels', type=int,
                        default=3, help='num of init channels')
    parser.add_argument('--layers', type=int, default=5,
                        help='total number of layers')
    parser.add_argument('--model_path', type=str,
                        default='saved_models', help='path to save the model')
    parser.add_argument('--cutout', action='store_true',
                        default=False, help='use cutout')
    parser.add_argument('--cutout_length', type=int,
                        default=16, help='cutout length')
    parser.add_argument('--drop_path_prob', type=float,
                        default=0.3, help='drop path probability')
    parser.add_argument('--save', type=str, default='EXP',
                        help='experiment name')
    parser.add_argument('--seed', type=int, default=2, help='random seed')
    parser.add_argument('--grad_clip', type=float,
                        default=5, help='gradient clipping')
    parser.add_argument('--train_portion', type=float,
                        default=1.0, help='portion of training data')
    parser.add_argument('--unrolled', action='store_true',
                        default=False, help='use one-step unrolled validation loss')
    parser.add_argument('--arch_learning_rate', type=float,
                        default=3e-4, help='learning rate for arch encoding')
    parser.add_argument('--arch_weight_decay', type=float,
                        default=1e-3, help='weight decay for arch encoding')
    args = parser.parse_args()

    return args

# Train Function

In [5]:
def train(x_train, y_train, x_valid, y_valid, model, architect, optimizer):
    """Trains the network. Gradient step is performed here

    Args:
        train_queue (array): Train queue
        valid_queue (array): Validation queue
        model (Network): Network
        architect (Architect): the architechture of network
        criterion (fn): Loss function
        optimizer (Optimiser): Adam / SGD
        lr (float): Learning Rate

    Returns:
        (float, float): returns acc and miOu
    """

    # architect step
    architect_step = architect.step(input_train=x_train,
                                    target_train=y_train,
                                    input_valid=x_valid,
                                    target_valid=y_valid,
                                    unrolled=args.unrolled,
                                    )
    
    with tf.control_dependencies([architect_step]):
        with tf.GradientTape() as tape:
            logits = model(x_train)
            w_var = model.get_thetas()
            loss = model._criterion(logits, y_train)
            grads = tape.gradient(loss, w_var)
            clipped_gradients, norm = tf.clip_by_global_norm(grads, args.grad_clip)
            opt_op = optimizer.apply_gradients(zip(clipped_gradients, w_var))

    # calculating accuracy and iou
    acc = utils.accuracy(logits, y_train)
    iou = utils.iou(logits, y_train)

    return loss, acc, iou

## Train Test

In [6]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "arch_weight_decay": 1e-3,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate_min": 0.001,
    "learning_rate": 0.025,
    "unrolled": True,
    "epochs": 10,
    "batch_size": 4,
    "save": "EXP"
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

In [7]:
np_ds_train = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
np_ds_valid = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
ds_train = tf.data.Dataset.from_tensor_slices(np_ds_train).batch(4)
ds_valid = tf.data.Dataset.from_tensor_slices(np_ds_valid).batch(4)

In [8]:
criterion = tf.losses.sigmoid_cross_entropy
model = Network(3, 3, criterion)
optimizer = tf.train.MomentumOptimizer(args.learning_rate_min, args.momentum)

In [9]:
architect = Architect(model, args)

In [10]:
init = tf.global_variables_initializer()

# Infer

In [12]:
def infer(x_valid, y_valid, logits, model, criterion):
    loss_op = model._loss(logits, y_valid)
    acc_op = utils.accuracy(logits, y_valid)
    iou_op = utils.iou(logits, y_valid)
    return loss_op, acc_op, iou_op

# Main

In [55]:
def input_fn():

    image_dataset = tf.data.TFRecordDataset('../../datasets/infer/infer-00000-00007.tfrecords')
    W, H = 16, 16
    
    # Create a dictionary describing the features.  
    image_feature_description = {
        'name': tf.FixedLenFeature([], tf.string),  
        'label_encoded': tf.FixedLenFeature([], tf.string),
        'encoded': tf.FixedLenFeature([], tf.string)
    }
    
    def _parse_image_function(example_proto):
        # Parse the input tf.Example proto using the dictionary above.
        feature= tf.parse_single_example(example_proto, image_feature_description)
        image= feature['encoded']
        label = feature['label_encoded']
        name = feature['name']

        image = tf.image.decode_png(image, channels=3)
        label = tf.image.decode_png(label, channels=3)

        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (W, H))
        label = tf.cast(label, tf.float32)
        label = tf.image.resize(label, (W, H))

        return image, label

    dataset = image_dataset.map(_parse_image_function)
    dataset = dataset.batch(8)
    
    return dataset

In [62]:
def main(args):
#     np_ds_train = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
    np_ds_valid = (np.random.randint(0, 256, (20, 16, 16, 3)).astype(np.float32), np.random.randint(0, 2, (20, 16, 16, 1)).astype(np.float32))
#     ds_train = tf.data.Dataset.from_tensor_slices(np_ds_train).batch(args.batch_size)
#     ds_valid = tf.data.Dataset.from_tensor_slices(np_ds_valid).batch(args.batch_size)
    ds_train = input_fn()
    ds_valid = input_fn()
    num_iterations = int(np_ds_train[0].shape[0] / args.batch_size)

    criterion = tf.losses.sigmoid_cross_entropy
    model = Network(3, 3, criterion)

    # Optimizer
    optimizer = tf.train.MomentumOptimizer(args.learning_rate_min, args.momentum)

    architect = Architect(model, args)

    init = tf.global_variables_initializer()
    
    _logits = model(tf.convert_to_tensor(np_ds_train[0][:4]))
    mious = [0]
    for e in range(args.epochs):
        tf.logging.info('Epoch {}'.format(e))

        train_it = ds_train.make_one_shot_iterator()
        valid_it = ds_valid.make_one_shot_iterator()

        tq1 = tqdm(range(num_iterations))
        genotype = model.genotype()
        print('Genotype: {}'.format(genotype))

        # Train Loop
        train_miou = 0
        train_unions, train_intersections = [], []    

        for i in tq1:
            x_train, y_train = train_it.get_next()
            y_train = y_train[:,:,:,0]
            print(y_train.shape)
            y_train = tf.reshape(y_train, (8, 16, 16, 1))
            x_valid, y_valid = valid_it.get_next()
            y_valid = y_valid[:,:,:,0]
            y_valid = tf.reshape(y_valid, (8, 16, 16, 1))
            train_loss, train_acc, train_iou = train(x_train=x_train,
                                                     y_train=y_train,
                                                     x_valid=x_valid,
                                                     y_valid=y_valid,
                                                     model=model,
                                                     architect=architect,
                                                     optimizer=optimizer
                                                     )
            if(i % args.report_freq == 0):
                tq1.set_postfix({
                        "Train Loss": train_loss.numpy(),
                        "Train Acc": train_acc.numpy(),
                        "Train IoU": train_iou[0].numpy()
                        })
            train_intersections.append(train_iou[1].numpy())
            train_unions.append(train_iou[2].numpy())
    
        # Calculation of train miou
        train_unions = np.array(train_unions)
        train_intersections = np.array(train_intersections)
        train_non_zero_mask = train_unions != 0
        train_miou = np.mean(train_intersections[train_non_zero_mask])/(np.mean(train_unions[train_non_zero_mask]) + 1e-6)

        # Log train miou
        logging.info('Train mIoU: {}'.format(train_miou))

        # Validation loop
        valid_unions, valid_intersections = [], []
        tq2 = tqdm(range(num_iterations))
        for i in tq2:
            valid_logits = model(x_valid)
            valid_loss, valid_acc, valid_iou = infer(x_valid=x_valid, 
                                                       y_valid=y_valid,
                                                       logits=valid_logits,
                                                       model=model,
                                                       criterion=criterion
                                                       )

            if(i % args.report_freq == 0):
                tq2.set_postfix({
                                "Valid Loss": valid_loss.numpy(),
                                "Valid Acc": valid_acc.numpy(),
                                "Valid IoU": valid_iou[0].numpy()
                                })
            valid_intersections.append(valid_iou[1])  
            valid_unions.append(valid_iou[2])

        # Calculation of train miou
        valid_unions = np.array(valid_unions)
        valid_intersections = np.array(valid_intersections)
        valid_non_zero_mask = valid_unions != 0
        valid_miou = np.mean(valid_intersections[valid_non_zero_mask])/(np.mean(valid_unions[valid_non_zero_mask]) + 1e-6)

        #Log Miou
        logging.info('Validation mIoU: {}'.format(valid_miou))

        # save the final genotype
        if(max(mious) < valid_iou):
            logging.info("Writing the computed genotype tp ./cnn/final_models/final_genotype.py")
            utils.write_genotype(genotype)

        mious.append(valid_miou)
  
    np.save(os.path.join(args.save, "mIoUs.npy"), mious)

## Test

In [63]:
def parse_args():
    args = {
        "momentum": 0.9,
        "weight_decay": 3e-4,
        "arch_learning_rate": 3e-1,
        "arch_weight_decay": 1e-3,
        "momentum": 0.9,
        "grad_clip": 5,
        "learning_rate_min": 0.001,
        "learning_rate": 0.025,
        "unrolled": True,
        "epochs": 1,
        "batch_size": 4,
        "save": "EXP",
        "report_freq": 1
    }

    class Struct:
        def __init__(self, **entries):
            self.__dict__.update(entries)

    return Struct(**args)

In [64]:
if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.ERROR)
    search_dirs = glob.glob('./search-*')
    [shutil.rmtree(search_dir) for search_dir in search_dirs]
    
    args = parse_args()
    args.save = 'search-{}-{}'.format(args.save,
                                      time.strftime("%Y%m%d-%H%M%S"))
    utils.create_exp_dir(args.save, scripts_to_save=glob.glob('*.py'))
    
    # logging
    log_format = '%(asctime)s %(message)s'
    logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                        format=log_format, datefmt='%m/%d %I:%M:%S %p')
    fh = logging.FileHandler(os.path.join(args.save, 'log.txt'))
    fh.setFormatter(logging.Formatter(log_format))
    logging.getLogger().addHandler(fh)
    

    main(args)

Experiment dir : search-EXP-20191109-193342







  0%|          | 0/5 [00:00<?, ?it/s]

Genotype: Genotype(normal=[('sep_conv_5x5', 0), ('sep_conv_5x5', 1), ('sep_conv_3x3', 1), ('sep_conv_3x3', 0), ('sep_conv_3x3', 0), ('dil_conv_5x5', 3), ('sep_conv_5x5', 3), ('dil_conv_3x3', 2)], normal_concat=[2, 3, 4, 5], reduce=[('sep_conv_5x5', 1), ('avg_pool_3x3', 0), ('dil_conv_5x5', 1), ('max_pool_3x3', 0), ('sep_conv_3x3', 0), ('sep_conv_3x3', 1), ('sep_conv_5x5', 4), ('max_pool_3x3', 1)], reduce_concat=[2, 3, 4, 5])
(8, 16, 16)







  0%|          | 0/5 [00:26<?, ?it/s, Train Acc=0.00146, Train Loss=3.25, Train IoU=0.00147]




 20%|██        | 1/5 [00:26<01:44, 26.01s/it, Train Acc=0.00146, Train Loss=3.25, Train IoU=0.00147]

(8, 16, 16)


...

(8, 16, 16)







 80%|████████  | 4/5 [02:10<00:25, 25.73s/it, Train Acc=0.819, Train Loss=-8.91, Train IoU=0.821]




100%|██████████| 5/5 [02:10<00:00, 26.05s/it, Train Acc=0.819, Train Loss=-8.91, Train IoU=0.821]

11/09 07:36:00 PM Train mIoU: 0.495078739914








  0%|          | 0/5 [00:00<?, ?it/s]




  0%|          | 0/5 [00:01<?, ?it/s, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 20%|██        | 1/5 [00:01<00:06,  1.74s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 20%|██        | 1/5 [00:03<00:06,  1.74s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 40%|████      | 2/5 [00:03<00:05,  1.74s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 40%|████      | 2/5 [00:05<00:05,  1.74s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 60%|██████    | 3/5 [00:05<00:03,  1.74s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 60%|██████    | 3/5 [00:06<00:03,  1.74s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 80%|████████  | 4/5 [00:06<00:01,  1.73s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




 80%|████████  | 4/5 [00:08<00:01,  1.73s/it, Valid Acc=0.833, Valid Loss=-13.3, Valid IoU=0.833]




100%|██████████| 5/5 [00:08<00:00,  1.73s/

11/09 07:36:09 PM Validation mIoU: 0.832926233106
11/09 07:36:09 PM Writing the computed genotype tp ./cnn/final_models/final_genotype.py


In [14]:
ds = input_fn()
it = ds.make_one_shot_iterator()


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [16]:
res = model(it.get_next()[0])

In [73]:
tf.nn.sigmoid(res[0])

<tf.Tensor: id=1540007, shape=(16, 16, 1), dtype=float32, numpy=
array([[[0.43516126],
        [0.5292279 ],
        [0.3452683 ],
        [0.48732194],
        [0.31998283],
        [0.47137895],
        [0.30243063],
        [0.456796  ],
        [0.30057225],
        [0.4411892 ],
        [0.3137334 ],
        [0.45296952],
        [0.32190293],
        [0.4592965 ],
        [0.3664359 ],
        [0.5236181 ]],

       [[0.43221235],
        [0.4170961 ],
        [0.37205014],
        [0.37814733],
        [0.35838842],
        [0.3655147 ],
        [0.35138226],
        [0.36091548],
        [0.34425634],
        [0.3547306 ],
        [0.3535769 ],
        [0.35948607],
        [0.34859842],
        [0.3001175 ],
        [0.34961835],
        [0.45303375]],

       [[0.3317604 ],
        [0.5274001 ],
        [0.28365847],
        [0.47794035],
        [0.24589646],
        [0.46459946],
        [0.22095093],
        [0.44219023],
        [0.2179637 ],
        [0.42034268],
       

In [27]:
res.numpy().sum(axis=0).shape

(16, 16, 1)

In [33]:
# Understand the softmax working

In [77]:
tf.round(tf.sigmoid(res)[0].numpy()*6)

<tf.Tensor: id=1540024, shape=(16, 16, 1), dtype=float32, numpy=
array([[[3.],
        [3.],
        [2.],
        [3.],
        [2.],
        [3.],
        [2.],
        [3.],
        [2.],
        [3.],
        [2.],
        [3.],
        [2.],
        [3.],
        [2.],
        [3.]],

       [[3.],
        [3.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [3.]],

       [[2.],
        [3.],
        [2.],
        [3.],
        [1.],
        [3.],
        [1.],
        [3.],
        [1.],
        [3.],
        [1.],
        [3.],
        [1.],
        [2.],
        [1.],
        [3.]],

       [[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [3.]],

       [[2.],
        [3.],
   

In [25]:
res[0]

<tf.Tensor: id=48603, shape=(16, 16, 1), dtype=float32, numpy=
array([[[-0.2608236 ],
        [ 0.11704504],
        [-0.6399038 ],
        [-0.05072308],
        [-0.7538507 ],
        [-0.11460948],
        [-0.8357501 ],
        [-0.17324805],
        [-0.84457445],
        [-0.23633718],
        [-0.7827227 ],
        [-0.1886797 ],
        [-0.7450404 ],
        [-0.1631751 ],
        [-0.5475378 ],
        [ 0.09454274]],

       [[-0.2728305 ],
        [-0.3347057 ],
        [-0.5234318 ],
        [-0.49741936],
        [-0.58236575],
        [-0.5515077 ],
        [-0.6129689 ],
        [-0.571393  ],
        [-0.64438343],
        [-0.5983093 ],
        [-0.60335326],
        [-0.5775955 ],
        [-0.62520576],
        [-0.8467386 ],
        [-0.6207173 ],
        [-0.18842053]],

       [[-0.7002338 ],
        [ 0.10971034],
        [-0.9263861 ],
        [-0.08829594],
        [-1.120619  ],
        [-0.1418395 ],
        [-1.2601333 ],
        [-0.23227787],
        [-1.2